In [1]:
%reload_ext sql
%sql sqlite:///test_capital_project.db

'Connected: @test_capital_project.db'

**Basic SQL Reminder Notes**

SELECT *

FROM *

WHERE *

"%" is wildcard for strings

"XOR" for one or other, but not both

ORDER BY [column name] asc/desc (asc/desc can be used after individual columns

ROUND works as we understand
--------

IN for lists Example:
-----
SELECT name FROM bbc
  WHERE name IN ('Sri Lanka', 'Ceylon',
                 'Persia',    'Iran')


**Multiple, indepnedent clauses are possible. Note the parentheses**

SELECT * FROM nobel

  WHERE subject IN ('physics')
  
  AND yr IN (1980)
  
  OR
  
  (subject IN ('chemistry')
  AND yr IN (1984))


In [ ]:
%%sql
CREATE TABLE balance_sheets (
    unit TEXT,
    assets REAL,
    liabilities REAL
);